# COLLABORATIVE  FILTERING - Recommending Books and Movies

We'll start by loading up the Goodreads dataset. Using Pandas, we can very quickly load the rows of the rating and item files that we care about, and merge them together so we can work with book names instead of ID's. (In a real production job, you'd stick with ID's and worry about the names at the display layer to make things more efficient. But this lets us understand what's going on better for now.)

In [79]:
import warnings
warnings.simplefilter('ignore')

In [80]:
import pandas as pd
import numpy as np


#### Illustration of the Principle

In [81]:
import numpy as np
np.random.seed(5)
ratings = np.random.randint(5, size=(5, 5), )
ratings

array([[3, 0, 1, 0, 4],
       [3, 0, 0, 4, 1],
       [0, 3, 4, 3, 1],
       [4, 2, 1, 1, 2],
       [1, 1, 1, 2, 0]])

In [82]:
ratingsDF = pd.DataFrame(ratings, columns=['BK1', 'BK2', 'BK3', 'BK4', 'BK5'])

In [83]:
ratingsDF['user'] = ['user1', 'user2', 'user3', 'user4', 'user5']
ratingsDF.set_index('user', inplace=True)

In [84]:
ratingsDF

,BK1,BK2,BK3,BK4,BK5
user,,,,,
user1,3,0,1,0,4
user2,3,0,0,4,1
user3,0,3,4,3,1
user4,4,2,1,1,2
user5,1,1,1,2,0


Pearson's correlation formula for a sample population: <br>

$r_{xy} = \frac{\sum_{i=1}^{n} (x_i - \bar{x}) (y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2 (y_i - \bar{y})^2}}$

In [85]:
corrTable = ratingsDF.corr()

In [86]:
ratingsDF[['BK2', 'BK3']]

,BK2,BK3
user,,
user1,0,1
user2,0,0
user3,3,4
user4,2,1
user5,1,1


In [87]:
corrTable['BK2']['BK3']

0.8344408667498864

In [88]:
ratingsDF[['BK4', 'BK5']]

,BK4,BK5
user,,
user1,0,4
user2,4,1
user3,3,1
user4,1,2
user5,2,0


In [89]:
corrTable['BK4']['BK5']

-0.7298004491997616

In [90]:
ratingsDF[['BK2', 'BK4']]

,BK2,BK4
user,,
user1,0,0
user2,0,4
user3,3,3
user4,2,1
user5,1,2


In [91]:
corrTable['BK2']['BK4']

0.12126781251816648

<div class="alert alert-info"> 

<h3> Hypothesis of a collaborative filtering based recommendation system.</h3>

<h4> 1. If the ratings of two items I1 and I2 are similar, i.e. correlation is a large +ve number, and a user likes I1, then user will like I2. </h4>

<h4> 2. If the ratings of two items I1 and I2 are opposing, i.e. correlation is a large -ve number, and a user likes I1, then user will dislike I2. </h4>

<h4> 3. If the ratings of two items I1 and I2 do not have any pattern, i.e. correlation is a small number, then it is difficult to conclude anything based on user liking I1.</h4>
</div>

In [92]:
corrTable = ratingsDF.corr()
corrTable

,BK1,BK2,BK3,BK4,BK5
BK1,1.000000,-0.490098,-0.742379,-0.384900,0.541736
BK2,-0.490098,1.000000,0.834441,0.121268,-0.328719
BK3,-0.742379,0.834441,1.000000,0.104257,-0.130435
BK4,-0.384900,0.121268,0.104257,1.000000,-0.729800
BK5,0.541736,-0.328719,-0.130435,-0.729800,1.000000


###  Load the data set of Movie Ratings from MovieLens source

In [93]:
# ratings=pd.read_csv('movies/ml-1m/ratings.dat', sep='::', header=None)
# ratings.columns = ['userId', 'movieId', 'rating', 'timeStamp']

corrTableName = "moviesCorrTable.pkl"
pathToRatings = '/Data/movieRatings.csv'
pathToDetails = '/Data/movieInfo.csv'

In [94]:
ratings = pd.read_csv(pathToRatings)
#ratings.drop('Unnamed: 0', axis=1, inplace=True)
ratings.shape

(1000209, 3)

In [95]:
ratings.sample(n=5, random_state=10)

,userId,itemId,rating
511275,3154,1625,3
467574,2881,1079,5
982145,5926,3256,4
867130,5232,1968,3
880583,5319,2294,4


In [96]:
# movies=pd.read_csv('movies/ml-1m/movies.dat', sep='::', header=None)
# movies.columns = ['MovieID', 'Title', 'Genres']
items=pd.read_csv(pathToDetails)
#items.drop('Unnamed: 0', axis=1, inplace=True)
items.shape

(3883, 3)

In [98]:
items.head(20) #(n=5, random_state=4)

,itemId,title,details
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


### Find out how many users in total

In [99]:
ratings['userId'].unique().size

6040

### Find out how many movies in total

In [100]:
ratings['itemId'].unique().size

3706

# Build the Pivot Table

<div class="alert alert-warning">
<b> Example Pivot Table. </b>
<br>
<br>
<img src="pivTable.JPG"/>
</div>

In [101]:
pivotTable = ratings.pivot_table(index=['userId'],columns=['itemId'],values='rating')
pivotTable.shape

(6040, 3706)

### Inspect the head

In [102]:
pivotTable.sample(n=5)

itemId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userId,,,,,,,,,,,,,,,,,,,,,
2044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4898,4.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3555,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN


##  Find out the correlation matrix of all books with each other

In [103]:
computeCorr = False
if computeCorr == False:
    corrTable = pd.read_pickle(corrTableName)
else:
    corrTable = pivotTable.corr(min_periods=350)
    corrTable.to_pickle(corrTableName)

In [104]:
#View the corrtable

corrTable.head()

itemId,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
itemId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.187467,NaN,NaN,NaN,0.051097,NaN,NaN,NaN,0.143598,...,NaN,NaN,NaN,NaN,NaN,0.139323,NaN,NaN,NaN,NaN
2,0.187467,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
corrTable.shape

(3706, 3706)

In [106]:
def itemsFromIDs(items, IDlist):
    df = pd.DataFrame(columns=items.columns)
    for id in IDlist:
        item = items[items.itemId == id] 
        df = pd.concat([df, item], axis=0)
    
    df.reset_index(inplace=True, drop=True)
    return df

In [107]:
def relatedRecos(itemName):
    ItemID = items[items.title == itemName]["itemId"].iloc[0]
    my_corr=corrTable.loc[ItemID]

    top10 = my_corr.dropna().sort_values(ascending=False)[1:11]
    top10itemIDs = list(top10.index)

    top10Items = itemsFromIDs(items, top10itemIDs)
    
    return top10Items

In [108]:
itemName ='Goldfinger (1964)'
relatedRecos(itemName)

,itemId,title,details
0,2949,Dr. No (1962),Action
1,2948,From Russia with Love (1963),Action
2,10,GoldenEye (1995),Action|Adventure|Thriller
3,1287,Ben-Hur (1959),Action|Adventure|Drama
4,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
5,1198,Raiders of the Lost Ark (1981),Action|Adventure
6,1201,"Good, The Bad and The Ugly, The (1966)",Action|Western
7,3793,X-Men (2000),Action|Sci-Fi
8,474,In the Line of Fire (1993),Action|Thriller
9,2640,Superman (1978),Action|Adventure|Sci-Fi


In [109]:
itemName = "Lost World: Jurassic Park, The (1997)"
#itemName='Lion King, The (1994)'

top10Recos = relatedRecos(itemName)
top10Recos

,itemId,title,details
0,480,Jurassic Park (1993),Action|Adventure|Sci-Fi
1,1882,Godzilla (1998),Action|Sci-Fi
2,1562,Batman & Robin (1997),Action|Adventure|Crime
3,160,Congo (1995),Action|Adventure|Mystery|Sci-Fi
4,736,Twister (1996),Action|Adventure|Romance|Thriller
5,1831,Lost in Space (1998),Action|Sci-Fi|Thriller
6,434,Cliffhanger (1993),Action|Adventure|Crime
7,780,Independence Day (ID4) (1996),Action|Sci-Fi|War
8,1917,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller
9,196,Species (1995),Horror|Sci-Fi


In [110]:
itemName='Indiana Jones and the Temple of Doom (1984)'
relatedRecos(itemName)

# Harrison Ford (Indiana Jones), Michael Douglas (Jewel of the Nile)

,itemId,title,details
0,2405,"Jewel of the Nile, The (1985)",Action|Adventure|Comedy|Romance
1,1291,Indiana Jones and the Last Crusade (1989),Action|Adventure
2,2002,Lethal Weapon 3 (1992),Action|Comedy|Crime|Drama
3,3033,Spaceballs (1987),Comedy|Sci-Fi
4,1375,Star Trek III: The Search for Spock (1984),Action|Adventure|Sci-Fi
5,3638,Moonraker (1979),Action|Romance|Sci-Fi
6,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
7,2001,Lethal Weapon 2 (1989),Action|Comedy|Crime|Drama
8,1373,Star Trek V: The Final Frontier (1989),Action|Adventure|Sci-Fi
9,2989,For Your Eyes Only (1981),Action


In [111]:
itemName='Lion King, The (1994)'
relatedRecos(itemName)


,itemId,title,details
0,588,Aladdin (1992),Animation|Children's|Comedy|Musical
1,595,Beauty and the Beast (1991),Animation|Children's|Musical
2,2081,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
3,1022,Cinderella (1950),Animation|Children's|Musical
4,2080,Lady and the Tramp (1955),Animation|Children's|Comedy|Musical|Romance
5,1907,Mulan (1998),Animation|Children's
6,2085,101 Dalmatians (1961),Animation|Children's
7,587,Ghost (1990),Comedy|Romance|Thriller
8,1,Toy Story (1995),Animation|Children's|Comedy
9,2355,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [112]:
itemName = "Superman (1978)"

top10Recos = relatedRecos(itemName)
top10Recos

,itemId,title,details
0,2641,Superman II (1980),Action|Adventure|Sci-Fi
1,2642,Superman III (1983),Action|Adventure|Sci-Fi
2,1270,Back to the Future (1985),Comedy|Sci-Fi
3,2011,Back to the Future Part II (1989),Comedy|Sci-Fi
4,2054,"Honey, I Shrunk the Kids (1989)",Adventure|Children's|Comedy|Fantasy|Sci-Fi
5,1375,Star Trek III: The Search for Spock (1984),Action|Adventure|Sci-Fi
6,2407,Cocoon (1985),Comedy|Sci-Fi
7,1097,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
8,2406,Romancing the Stone (1984),Action|Adventure|Comedy|Romance
9,1721,Titanic (1997),Drama|Romance


In [113]:
itemName = "Back to the Future (1985)"
#itemName='Lion King, The (1994)'
relatedRecos(itemName)


,itemId,title,details
0,2011,Back to the Future Part II (1989),Comedy|Sci-Fi
1,2012,Back to the Future Part III (1990),Comedy|Sci-Fi|Western
2,2640,Superman (1978),Action|Adventure|Sci-Fi
3,2054,"Honey, I Shrunk the Kids (1989)",Adventure|Children's|Comedy|Fantasy|Sci-Fi
4,2420,"Karate Kid, The (1984)",Drama
5,2797,Big (1988),Comedy|Fantasy
6,2407,Cocoon (1985),Comedy|Sci-Fi
7,2716,Ghostbusters (1984),Comedy|Horror
8,587,Ghost (1990),Comedy|Romance|Thriller
9,2470,Crocodile Dundee (1986),Adventure|Comedy


In [114]:
itemName = 'Godfather, The (1972)'
relatedRecos(itemName)


,itemId,title,details
0,1221,"Godfather: Part II, The (1974)",Action|Crime|Drama
1,1213,GoodFellas (1990),Crime|Drama
2,1263,"Deer Hunter, The (1978)",Drama|War
3,1997,"Exorcist, The (1973)",Horror
4,1084,Bonnie and Clyde (1967),Crime|Drama
5,1276,Cool Hand Luke (1967),Comedy|Drama
6,1953,"French Connection, The (1971)",Action|Crime|Drama|Thriller
7,2019,Seven Samurai (The Magnificent Seven) (Shichin...,Action|Drama
8,1954,Rocky (1976),Action|Drama
9,1208,Apocalypse Now (1979),Drama|War


In [115]:
itemName = 'Lawrence of Arabia (1962)'
relatedRecos(itemName)


,itemId,title,details
0,1953,"French Connection, The (1971)",Action|Crime|Drama|Thriller
1,1250,"Bridge on the River Kwai, The (1957)",Drama|War
2,1207,To Kill a Mockingbird (1962),Drama
3,1208,Apocalypse Now (1979),Drama|War
4,913,"Maltese Falcon, The (1941)",Film-Noir|Mystery
5,1233,"Boat, The (Das Boot) (1981)",Action|Drama|War
6,1262,"Great Escape, The (1963)",Adventure|War
7,1193,One Flew Over the Cuckoo's Nest (1975),Drama
8,924,2001: A Space Odyssey (1968),Drama|Mystery|Sci-Fi|Thriller
9,1276,Cool Hand Luke (1967),Comedy|Drama


In [116]:
itemName = 'Good, The Bad and The Ugly, The (1966)'
relatedRecos(itemName)


,itemId,title,details
0,2951,"Fistful of Dollars, A (1964)",Action|Western
1,2944,"Dirty Dozen, The (1967)",Action|War
2,1266,Unforgiven (1992),Western
3,1220,"Blues Brothers, The (1980)",Action|Comedy|Musical
4,1276,Cool Hand Luke (1967),Comedy|Drama
5,1208,Apocalypse Now (1979),Drama|War
6,2947,Goldfinger (1964),Action
7,3421,Animal House (1978),Comedy
8,1953,"French Connection, The (1971)",Action|Crime|Drama|Thriller
9,1968,"Breakfast Club, The (1985)",Comedy|Drama


#### Netflix Recommendation Engine (Much much more sophisticated)

1. Explicit behavior (like ratings given)
2. Looks at implicit user behavior (in how many sittings did you finish watching the movie, what time did you watch the movie)
3. Tagging of movie content (crime thriller, adventure, female leads etc etc). All of this is generated by human taggers.


#### Amazon recommendation engine.

1. Purchased shopping carts = real money from real people spent on real items = powerful data and a lot of it.
2. Items added to carts but abandoned.
3. Pricing experiments online (A/B testing, etc.) where they offer the same products at different prices and see the results
4. Wishlists - what's on them specifically for you - and in aggregate it can be treated similarly to another stream of basket analysis data
5. Referral sites (identification of where you came in from can hint other items of interest)
6. Dwell times (how long before you click back and pick a different item)
7. Ratings by you or those in your social network/buying circles - if you rate things you like you get more of what you like and if you confirm with the "i already own it" button they create a very complete profile of you
8. Demographic information (your shipping address, etc.) - they know what is popular in your general area for your kids, yourself, your spouse, etc.
9. user segmentation = did you buy 3 books in separate months for a toddler? likely have a kid or more.. etc.
10. Direct marketing click through data - did you get an email from them and click through? They know which email it was and what you clicked through on and whether you bought it as a result.
11. Click paths in session - what did you view regardless of whether it went in your cart
12. Number of times viewed an item before final purchase

#### Market Basket Analysis
See what is their in your basket and provide recommendations based on what people with such a basket might additionally buy.

1. For example, if basket has milk, then possibly egg and bread and cereal.
2. If basket has beer, then maybe peanuts etc/

### Try to find recommendations based on your favorite movies

In [ ]:
def searchForItems(items, searchStr):
    df = items[items['title'].str.contains(searchStr, case=False)]
    return list(df['title'])

In [ ]:
searchForItems(items, "rear")

In [ ]:
# Specify a movie which exists in the database as Item in the code below.

In [ ]:
itemName = 'Rear Window (1954)'

top10Recos = relatedRecos(itemName)
top10Recos

In [ ]:
itemName = 'My Fair Lady (1964)'

top10Recos = relatedRecos(itemName)
top10Recos

###  Load the data set of Book Ratings from Good Reads source

In [ ]:
# ratings=pd.read_csv('movies/ml-1m/ratings.dat', sep='::', header=None)
# ratings.columns = ['userId', 'movieId', 'rating', 'timeStamp']

corrTableName = "booksCorrTable.pkl"
pathToRatings = 'd:/ml/Data/booksV2/bookRatings.csv'
pathToDetails = 'd:/ml/Data/booksV2/bookInfo.csv'

In [ ]:
ratings = pd.read_csv(pathToRatings)

In [ ]:
ratings.head(n=5)

In [ ]:
# movies=pd.read_csv('movies/ml-1m/movies.dat', sep='::', header=None)
# movies.columns = ['MovieID', 'Title', 'Genres']
items=pd.read_csv(pathToDetails)

In [ ]:
items.sample(n=5)

### Find out how many users in total

In [ ]:
ratings['userId'].unique().size

### Find out how many movies in total

In [ ]:
items.shape[0]

# Build the Pivot Table

In [ ]:
pivotTable = ratings.pivot_table(index=['userId'],columns=['itemId'],values='rating')
pivotTable.shape

### Inspect the head

In [ ]:
pivotTable.sample(n=5)

##  Find out the correlation matrix of all books with each other

In [ ]:
%%time
computeCorr = False
if computeCorr == False:
    corrTable = pd.read_pickle(corrTableName)
else:
    corrTable = pivotTable.corr(min_periods=250)
    corrTable.to_pickle(corrTableName)

In [ ]:
#View the corrtable
corrTable.head()

In [ ]:
itemName = 'Harry Potter and the Deathly Hallows' 

top10Recos = relatedRecos(itemName)
top10Recos

In [ ]:
itemName = 'Twilight (Twilight, #1)'

top10Recos = relatedRecos(itemName)
top10Recos

In [ ]:
itemName = 'The Kite Runner'

top10Recos = relatedRecos(itemName)
top10Recos

In [ ]:
itemName = 'The Hunger Games'

top10Recos = relatedRecos(itemName)
top10Recos

In [ ]:
itemName = 'Divergent (Divergent, #1)'

top10Recos = relatedRecos(itemName)
top10Recos

### Try to find recommendations based on your favorite books

In [ ]:
searchForItems(items, "Gatsby")

In [ ]:
# Specify a movie which exists in the database as Item in the code below.

In [ ]:
itemName = 'The Great Gatsby'

top10Recos = relatedRecos(itemName)
top10Recos

In [ ]:
searchForItems(items, "Pride")

In [ ]:
itemName = ''

top10Recos = relatedRecos(itemName)
top10Recos